In [1]:
import os
import sys
import gzip
import numpy as np
import pandas as pd
import urllib.request
import umap.umap_ as umap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D

In [2]:
#!/usr/bin/python3

import os
import sys
import gzip
import numpy as np
import pandas as pd

txipIn= "/Users/AdrianHanson/Downloads/txip_fungi (1).npy"
txidxIn= "/Users/AdrianHanson/Downloads/ipromerge_fungi.txt";
ipidxIn= "/Users/AdrianHanson/Downloads/ipromerge_fungi.txt";

# interpro domain info
# https://ftp.ebi.ac.uk/pub/databases/interpro/current_release/entry.list

# tax id info ( ... with gzip.open(file_name, "rt") as ... )
#   proteomes_AND_proteome_type_1_AND_super_2023_09_20.tsv.gz
#   proteomes_AND_superkingdom_Bacteria_AND_2023_09_20.tsv.gz
#   proteomes_AND_Eucaryota_AND_super_2023_11_01.tsv.gz

txid = []
txidx = {}

ipid =[]
ipidx = {}

with open(txidxIn,"r") as fh:
    for ln in fh:
        if ln.startswith("tx:"):
            col = ln.strip().split()
            txid.append( col[1].strip() )
            txidx[ col[1].strip() ] = len(txid)-1

with open(ipidxIn,"r") as fh:
    for ln in fh:
        if ln.startswith("ipd:"):
            col = ln.strip().split()
            ipid.append(col[1].strip())
            ipidx[ col[1].strip() ] = len(ipid)-1

print( "tax:", len(txid), txid[:10] )
print( "dom:", len(ipid), ipid[:10] )

# numpy array
#------------

txip = np.load( txipIn )

print( txip.shape )
print( txip )
print( txip[:10,:10] )


# pandas dataframe
#-----------------

tipd = pd.DataFrame( txip, index= txidx, columns=ipidx )
print(tipd.shape)
print(tipd)

tax: 886 ['237561', '284813', '5544', '214684', '284812', '559292', '367110', '330879', '341663', '660122']
dom: 17143 ['IPR036815', 'IPR023410', 'IPR000308', 'IPR023409', 'IPR036441', 'IPR001874', 'IPR018509', 'IPR014710', 'IPR010329', 'IPR011051']
(886, 17143)
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [2 1 2 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]
[[1 1 1 1 1 1 1 2 1 2]
 [1 1 1 0 0 0 0 1 0 1]
 [2 1 2 1 1 1 1 2 1 2]
 [1 1 1 1 0 0 0 2 1 2]
 [1 1 1 1 0 0 0 1 0 1]
 [1 1 1 1 0 0 0 1 1 1]
 [2 2 2 1 1 1 1 2 1 2]
 [1 1 1 1 1 1 1 2 1 2]
 [1 1 1 1 1 1 1 2 1 2]
 [1 1 1 1 1 1 1 1 1 1]]
(886, 17143)
         IPR036815  IPR023410  IPR000308  IPR023409  IPR036441  IPR001874  \
237561           1          1          1          1          1          1   
284813           1          1          1          0          0          0   
5544             2          1          2          1          1          1   
214684           1          1          1          1          0   

In [3]:
# TODO: not sure if this entry_list is right
entry_list = pd.read_csv(r'/Users/AdrianHanson/Downloads/entry_list.txt', sep = '\t')
entry_list.head()

,ENTRY_AC,ENTRY_TYPE,ENTRY_NAME
0,IPR000126,Active_site,"Serine proteases, V8 family, serine active site"
1,IPR000138,Active_site,"Hydroxymethylglutaryl-CoA lyase, active site"
2,IPR000169,Active_site,"Cysteine peptidase, cysteine active site"
3,IPR000180,Active_site,"Membrane dipeptidase, active site"
4,IPR000189,Active_site,"Prokaryotic transglycosylase, active site"


In [4]:
with gzip.open("/Users/AdrianHanson/Desktop/Lab Files/Fungi Taxonomy/proteomes_AND_Eucaryota_AND_super_2023_11_01 (1).tsv.gz", 'r') as f:
    string_data = f.read().decode()
    list_data = [row.split("\t") for row in string_data.split("\n")]
    df_entries_eukaryota = pd.DataFrame(list_data[1:], columns=list_data[0])
#create df for all eukaryotes
#organism_organismid_df_archea = df_entries_archea.iloc[:, 1:3]
organism_organismid_df_eukaryota = pd.concat([df_entries_eukaryota['Organism Id'], df_entries_eukaryota['Organism']], axis=1)
df_entries_eukaryota.head()

,Proteome Id,Organism,Organism Id,Protein count,BUSCO,CPD,Taxon mnemonic,Taxonomic lineage
0,UP000002311,Saccharomyces cerevisiae (strain ATCC 204508 /...,559292,6060,"C:99.6%[S:97.4%,D:2.2%],F:0.1%,M:0.3%,n:2137",Outlier (high value),YEAST,"Eukaryota, Opisthokonta, Fungi, Dikarya, Ascom..."
1,UP000000589,Mus musculus (Mouse),10090,55087,"C:99.8%[S:51.2%,D:48.6%],F:0.0%,M:0.2%,n:13798",Outlier (high value),MOUSE,"Eukaryota, Opisthokonta, Metazoa, Eumetazoa, B..."
2,UP000000803,Drosophila melanogaster (Fruit fly),7227,22061,"C:100.0%[S:41.8%,D:58.2%],F:0.0%,M:0.0%,n:3285",Outlier (high value),DROME,"Eukaryota, Opisthokonta, Metazoa, Eumetazoa, B..."
3,UP000002494,Rattus norvegicus (Rat),10116,47942,"C:98.1%[S:45.5%,D:52.6%],F:0.3%,M:1.6%,n:13798",Outlier (high value),RAT,"Eukaryota, Opisthokonta, Metazoa, Eumetazoa, B..."
4,UP000005640,Homo sapiens (Human),9606,82678,"C:99.5%[S:37.4%,D:62.1%],F:0.0%,M:0.5%,n:13780",Outlier (high value),HUMAN,"Eukaryota, Opisthokonta, Metazoa, Eumetazoa, B..."


In [5]:
#get ids for all eukaryota in original table from tipd
row_names = list(tipd.index.values.tolist())

eukaryota_ids = df_entries_eukaryota['Organism Id']
eukaryota_ids = eukaryota_ids.to_list()
subset_ids_eukaryota = []
eukaryota = []
for i in range(len(row_names)):
    if row_names[i] in eukaryota_ids:
        subset_ids_eukaryota.append(i)
        eukaryota.append("Eukaryota")
    else: eukaryota.append("N/A")


# print(eukaryota_ids)


In [6]:
print(None in eukaryota_ids)

True


In [7]:
#global organism_organismid
organism_organismid = pd.DataFrame(organism_organismid_df_eukaryota)
organism_organismid.head()
organism_organismid.set_index('Organism Id',inplace=True)


In [8]:
organism_organismid.columns

Index(['Organism'], dtype='object')

In [9]:
eukaryota_ids = [str(i) for i in df_entries_eukaryota['Organism Id'].to_list()]



In [10]:
def extract_taxonomic_ranks(df, levels=15):
    # Split the 'Taxonomic lineage' by ', ' and expand to the desired number of columns
    tax_ranks = df['Taxonomic lineage'].str.split(', ', expand=True)

    # Ensure there are enough columns, filling with NA if necessary
    for _ in range(tax_ranks.shape[1], levels):
        tax_ranks[_] = np.nan  # Add a new column filled with NaNs

    # Take only the first `levels` columns
    tax_ranks = tax_ranks.iloc[:, :levels]

    # Name the columns appropriately
    tax_ranks.columns = ['Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species','depth 7', 'depth 8',
                        'depth 9','depth 10','depth 11','depth 12','depth 13','depth 14']

    # Replace empty strings with NaN
    tax_ranks.replace('', np.nan, inplace=True)

    # Set the "Organism Id" as the index
    tax_ranks.index = df['Organism Id']

    return tax_ranks

archaea_tax_ranks = extract_taxonomic_ranks(df_entries_eukaryota)
bacteria_tax_ranks = extract_taxonomic_ranks(df_entries_eukaryota)

df_combined = pd.concat([archaea_tax_ranks, bacteria_tax_ranks], ignore_index=False)
df_combined = df_combined[~df_combined.index.isna()]
df_combined = df_combined[df_combined.index.isin(tipd.index)]
df_combined

,Domain,Phylum,Class,Order,Family,Genus,Species,depth 7,depth 8,depth 9,depth 10,depth 11,depth 12,depth 13,depth 14
Organism Id,,,,,,,,,,,,,,,
559292,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,saccharomyceta,Saccharomycotina,Saccharomycetes,Saccharomycetales,Saccharomycetaceae,Saccharomyces,Saccharomyces cerevisiae,None,None,None
284812,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,Taphrinomycotina,Schizosaccharomycetes,Schizosaccharomycetales,Schizosaccharomycetaceae,Schizosaccharomyces,Schizosaccharomyces pombe,None,None,None,None
237561,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,saccharomyceta,Saccharomycotina,Saccharomycetes,Saccharomycetales,CUG-Ser1 clade,Debaryomycetaceae,Candida/Lodderomyces clade,Candida,Candida albicans,None
436907,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,saccharomyceta,Saccharomycotina,Saccharomycetes,Saccharomycetales,Saccharomycetaceae,Vanderwaltozyma,Vanderwaltozyma polyspora,None,None,None
644223,Eukaryota,Opisthokonta,Fungi,Dikarya,Ascomycota,saccharomyceta,Saccharomycotina,Saccharomycetes,Saccharomycetales,Phaffomycetaceae,Komagataella,Komagataella phaffii,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4874,Eukaryota,Opisthokonta,Fungi,Fungi incertae sedis,Mucoromycota,Glomeromycotina,Glomeromycetes,Diversisporales,Gigasporaceae,Gigaspora,None,None,None,None,None
980116,Eukaryota,Opisthokonta,Fungi,Dikarya,Basidiomycota,Agaricomycotina,Agaricomycetes,Agaricomycetidae,Agaricales,Agaricineae,Psathyrellaceae,Tulosesus,None,None,None
2733690,Eukaryota,Opisthokonta,Fungi,Dikarya,Basidiomycota,Agaricomycotina,Agaricomycetes,Agaricomycetidae,Agaricales,Marasmiineae,Mycenaceae,Mycena,None,None,None


In [46]:
counter=0

for index, row in df_combined.iterrows():
    if row[4] == 'Mucoromycota':
        counter+=1

print(f'Mucoromycota is {counter/1772}% of the dataset')

Mucoromycota is 0.04288939051918736% of the dataset


In [25]:
#create supervised UMAP
def sup_UMAP(data, tax_ranks, names, neighbors=False, in_metric='cosine', k=12, depth=5):
    
    #the idea here is to get the data shapes to line up so that we can do analysis
    species_w_proteomes = pd.DataFrame()
    # Filter species_w_proteomes to contain only rows present in both data and tax_ranks
    species_w_proteomes = tax_ranks[tax_ranks.index.isin(data.index)]

    # Check if there are rows remaining in species_w_proteomes
    if species_w_proteomes.empty:
        print("No matching species found in tax_ranks DataFrame.")
        return
    #create list of labels, and encode numerical values 


    classes = []
    for i in range(species_w_proteomes.shape[0]):
        if species_w_proteomes.iloc[i,depth] == None:
            class_label = f'No label at depth {depth}'
            species_w_proteomes.iloc[i,depth] = class_label
            if class_label not in classes:
                classes.append(class_label)
            continue

        if species_w_proteomes.iloc[i,depth] not in classes:
            classes.append(species_w_proteomes.iloc[i,depth])

    
    
    if depth < 1:
        if neighbors is True:
            embedding = umap.UMAP(n_neighbors= k, n_components= 2, metric=in_metric, init='random',random_state=42).fit_transform(X=data)
        else:
            embedding = umap.UMAP(n_components= 2, init='random', metric=in_metric, random_state=42).fit_transform(X=data)
        target = None
              #plotly plot
        fig = px.scatter(embedding, x = 0, y= 1,
                        color = species_w_proteomes.iloc[:,4],
                        hover_name = names,
                        title= 'Unsupervised UMAP: KNN = 12')
        fig.show()
    else:
        
        #encode all values
        encoder = LabelEncoder()
        labels = encoder.fit_transform(species_w_proteomes.iloc[:,depth])   
      
        
        #create UMAP embedding
        if neighbors is True:
            reducer = umap.UMAP(n_neighbors= k, n_components=2, init='random',metric=in_metric,random_state=42)
        else:
            reducer = umap.UMAP(n_components=2, init='random',metric=in_metric, random_state=42)


        embedding= reducer.fit_transform(X= data, y=labels)

       #plotly plot
        fig = px.scatter(embedding, x = 0, y= 1,
                        color = species_w_proteomes.iloc[:,depth],
                        hover_name = names,
                        title= f'Supervised UMAP by {tax_ranks.columns[depth]}: KNN = 12')
        fig.show()
    

    

In [26]:
protein_data = tipd.loc[:,tipd.columns != 'Phylum']
print("protein data shape: ", protein_data.shape)
print("df_combined shape: ",df_combined.shape)

yes_proteome = pd.DataFrame()
for specimen in protein_data.index:
    if specimen in df_combined.index:
        yes_proteome = pd.concat([yes_proteome, df_combined.loc[specimen]])
        
fungi_organism_organismid = pd.DataFrame()
for organism in protein_data.index:
    if organism in organism_organismid.index:
        if organism_organismid.loc[organism]['Organism'] == None:
            print(organism_organismid.loc[organism]['Organism'])
        fungi_organism_organismid = pd.concat([fungi_organism_organismid, organism_organismid.loc[[organism]]])
        
        
yes_proteome = yes_proteome[~yes_proteome.index.duplicated()]
fungi_organism_organismid = fungi_organism_organismid[~fungi_organism_organismid.index.duplicated()]

#GET ONLY NAMES FROM ABOVE LIST

organism_names = fungi_organism_organismid['Organism'].tolist()

# print("yes_proteome shape ", yes_proteome.shape)
print("fungi_organism_organismid shape ", fungi_organism_organismid.shape)
print(fungi_organism_organismid.isnull().any())

protein data shape:  (886, 17143)
df_combined shape:  (1772, 15)
fungi_organism_organismid shape  (886, 1)
Organism    False
dtype: bool


In [35]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 0, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [28]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 2, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [29]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 3, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [30]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 4, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [31]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 5, k= 12, names= organism_names)


/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [32]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 6, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [33]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 7, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [34]:
sup_UMAP(data = protein_data, tax_ranks=yes_proteome, neighbors = True, depth= 8, k= 12, names= organism_names)

/Users/AdrianHanson/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

